In [39]:

import pandas as pd



In [40]:
# Load the datasets
Stripe_payment = pd.read_excel(r"C:\Users\cryst\Desktop\Stripe DS Written Project Data 2022-20240827T030341Z-001\Stripe DS Written Project Data 2022\dstakehome_payments.xlsx")
Stripe_merchants = pd.read_excel(r"C:\Users\cryst\Desktop\Stripe DS Written Project Data 2022-20240827T030341Z-001_1\Stripe DS Written Project Data 2022\dstakehome_merchants.xlsx")
Stripe_payment.info()
Stripe_merchants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1577887 entries, 0 to 1577886
Data columns (total 6 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   date                 1577887 non-null  object
 1   merchant             1577887 non-null  object
 2   subscription_volume  1577887 non-null  int64 
 3   checkout_volume      1577887 non-null  int64 
 4   payment_link_volume  1577887 non-null  int64 
 5   total_volume         1577887 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 72.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23627 entries, 0 to 23626
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   merchant           23627 non-null  object
 1   industry           23627 non-null  object
 2   first_charge_date  23627 non-null  object
 3   country            23627 non-null  object
 4   business_size      23627 non-null 

In [43]:
#Convert date to datetime, with errors converted to NaT
Stripe_merchants['first_charge_date'] = pd.to_datetime(Stripe_merchants['first_charge_date'], errors='coerce')

Stripe_payment['date'] = pd.to_datetime(Stripe_payment['date'], errors='coerce')

In [45]:
# Check for NaT values (invalid dates)
invalid_dates_count = Stripe_merchants['first_charge_date'].isna().sum()
invalid_dates_count1 = Stripe_payment['date'].isna().sum()

# Display the number of invalid dates in Stripe_merchants
print(f"Number of invalid dates converted to NaT in Stripe_merchants: {invalid_dates_count}")

# Display the number of invalid dates in Stripe_payment
print(f"Number of invalid dates converted to NaT in Stripe payment: {invalid_dates_count1}")

Number of invalid dates converted to NaT in Stripe_merchants: 26
Number of invalid dates converted to NaT in Stripe payment: 0


In [46]:
# Check for missing values in both datasets
payments_missing = Stripe_payment.isnull().sum()
merchants_missing = Stripe_merchants.isnull().sum()

print(payments_missing) 

print(merchants_missing)

date                   0
merchant               0
subscription_volume    0
checkout_volume        0
payment_link_volume    0
total_volume           0
dtype: int64
merchant              0
industry              0
first_charge_date    26
country               0
business_size         0
dtype: int64


In [49]:
 #Save the missing values to an Excel file for traceability and future-proofing your analysis
merchants_missing.to_excel('missing_values_Stripe_merchants.xlsx', index=False)
# Drop rows with NaT in critical date fields
Stripe_merchants = Stripe_merchants.dropna(subset=['first_charge_date'])

# Converts USD cents to dollars by dividing each relevant column by 100
Stripe_payment['subscription_volume_usd'] = Stripe_payment['subscription_volume'] / 100
Stripe_payment['checkout_volume_usd'] = Stripe_payment['checkout_volume'] / 100
Stripe_payment['payment_link_volume_usd'] = Stripe_payment['payment_link_volume'] / 100
Stripe_payment['total_volume_usd'] = Stripe_payment['total_volume'] / 100



In [50]:
# Merge the datasets on the 'merchant' identifier
Stripe_merged = pd.merge(Stripe_payment, Stripe_merchants, on='merchant', how='inner')

In [63]:
# Filter merchants with subscription_volume_usd == 0 (new-to-Subscriptions)
non_subscription_users = Stripe_merged[Stripe_merged['subscription_volume_usd'] == 0]

In [64]:
# Identify potential users for Subscriptions
potential_subscription_users = non_subscription_users[
    (non_subscription_users['checkout_volume_usd'] > 0) | 
    (non_subscription_users['payment_link_volume_usd'] > 0)
].copy()  # Add .copy() here to create a copy of the DataFrame slice


In [65]:
# Calculate rank score using .loc 
potential_subscription_users.loc[:, 'rank_score'] = (
    potential_subscription_users['checkout_volume_usd'] +
    potential_subscription_users['payment_link_volume_usd']
)
# Sort the merchants by the rank_score
potential_subscription_users = potential_subscription_users.sort_values(by='rank_score', ascending=False)


In [66]:
# Select relevant columns for the final output
target_columns = ['merchant', 'industry', 'country', 'business_size', 'first_charge_date', 'checkout_volume_usd', 'payment_link_volume_usd', 'rank_score']
potential_subscription_users = potential_subscription_users[target_columns]

In [67]:
# Save the target merchants list to a CSV file
potential_subscription_users.to_csv('target_merchants_for_subscription_campaign.csv', index=False)

In [68]:
display(potential_subscription_users.head(10))

,merchant,industry,country,business_size,first_charge_date,checkout_volume_usd,payment_link_volume_usd,rank_score
1289402,cb905985,Ticketing & events,BR,small,2041-05-27 00:00:00+00:00,342590.69,0.0,342590.69
1332164,dbca5427,Clothing & accessory,US,medium,2041-06-15 00:00:00+00:00,315767.15,0.0,315767.15
1076005,77bace1e,Education,GB,medium,2039-07-09 00:00:00+00:00,299804.14,0.0,299804.14
1332161,dbca5427,Clothing & accessory,US,medium,2041-06-15 00:00:00+00:00,291501.60,0.0,291501.60
214454,c939a25f,Ticketing & events,CA,small,2038-06-21 00:00:00+00:00,248335.46,0.0,248335.46
1289401,cb905985,Ticketing & events,BR,small,2041-05-27 00:00:00+00:00,248293.06,0.0,248293.06
1498461,937b993c,Digital goods,SE,small,2041-11-09 00:00:00+00:00,240000.00,0.0,240000.00
1076004,77bace1e,Education,GB,medium,2039-07-09 00:00:00+00:00,216865.75,0.0,216865.75
1075947,77bace1e,Education,GB,medium,2039-07-09 00:00:00+00:00,201631.71,0.0,201631.71
1332165,dbca5427,Clothing & accessory,US,medium,2041-06-15 00:00:00+00:00,181625.18,0.0,181625.18
